In [1]:
from cuml.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from ImageRecognizerML.CustomModel.CustomVoting import CustomVoting
from sklearn.metrics import classification_report, accuracy_score
import cupy as cp
import numpy as np
import pandas as pd

# Getting Dataset

In [4]:
DATASET_DIR = 'Datasets'

df = pd.read_csv(f'{DATASET_DIR}/cifar10.csv')
df2 = pd.read_csv(F'{DATASET_DIR}/cifar10_features.csv')

columns = df.columns[:-1]
columns2 = df2.columns[:-1]

scaler = StandardScaler()
df[columns] = scaler.fit_transform(df[columns])
df2[columns2] = scaler.fit_transform(df2[columns2])

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/cifar10_features.csv'

# Removing Outliers

In [12]:
X = df.iloc[:, :-1].astype(np.float64)
y = df.iloc[:, -1].astype(np.int8)
X2 = df2.iloc[:, :-1].astype(np.float64)
y2 = df2.iloc[:, -1].astype(np.int8)

lof = LocalOutlierFactor(n_neighbors=5, contamination=0.02, n_jobs=-1)
lof_outliers = lof.fit_predict(X)

outliers_idx = np.where(lof_outliers == -1)[0]

X = np.delete(X, outliers_idx, axis=0)
y = np.delete(y, outliers_idx, axis=0)
X2 = np.delete(X2, outliers_idx, axis=0)
y2 = np.delete(y2, outliers_idx, axis=0)

X = cp.array(X)
y = cp.array(y)
X2 = cp.array(X2)
y2 = cp.array(y2)

# Training Model

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_features, X_test_features, y_train_features, y_test_features = train_test_split(X2, y2, test_size=0.2, random_state=42)

model = CustomVoting(models=[
    XGBClassifier(n_estimators=900, learning_rate=0.3, max_depth=6, device='cuda'), 
    XGBClassifier(n_estimators=2000, learning_rate=0.2, max_depth=5, device='cuda')
], weights=[0.7, 0.3])
model.fit([{'X_train': X_train, 'y_train': y_train}, {'X_train': X_train_features, 'y_train': y_train_features}])

y_pred = model.predict([X_test, X_test_features])

print(accuracy_score(y_test.get(), y_pred))
print(classification_report(y_test.get(), y_pred))

0.6005952380952381
              precision    recall  f1-score   support

           0       0.64      0.63      0.64      1250
           1       0.69      0.69      0.69      1122
           2       0.51      0.48      0.49      1168
           3       0.45      0.44      0.45      1173
           4       0.52      0.49      0.50      1160
           5       0.52      0.50      0.51      1170
           6       0.62      0.71      0.66      1219
           7       0.66      0.65      0.65      1155
           8       0.72      0.76      0.74      1198
           9       0.64      0.67      0.65      1145

    accuracy                           0.60     11760
   macro avg       0.60      0.60      0.60     11760
weighted avg       0.60      0.60      0.60     11760

